In [1]:
using PLite

┌ Info: Precompiling PLite [7876af07-990d-54b4-8888-23690620f79a]
└ @ Base loading.jl:1317
┌ Warning: Package PLite does not have SharedArrays in its dependencies:
│ - If you have PLite checked out for development and have
│   added SharedArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PLite
└ Loading SharedArrays into PLite from project dependency, future warnings for PLite are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [2]:
# constants
const MinX = 0
const MaxX = 100
const StepX = 20

# mdp definition
mdp = MDP()

MDP(Dict{AbstractString, PLite.LazyVar}(), Dict{AbstractString, PLite.LazyVar}(), PLite.LazyFunc(true, String[], PLite.var"#emptyfunc#1"()), PLite.LazyFunc(true, String[], PLite.var"#emptyfunc#1"()))

In [3]:
# state space
statevariable!(mdp, "x", MinX, MaxX)  # continuous
statevariable!(mdp, "goal", ["no", "yes"])  # discrete

# action space
actionvariable!(mdp, "move", ["W", "E", "stop"])  # discrete

PLite.ValuesVar("move", ["W", "E", "stop"])

In [4]:
function isgoal(x::Float64)
  if abs(x - MaxX / 2) < StepX
    return "yes"
  else
    return "no"
  end
end

function mytransition(x::Float64, goal::AbstractString, move::AbstractString)
  if isgoal(x) == "yes" && goal == "yes"
    return [([x, isgoal(x)], 1.0)]
  end

  if move == "E"
    if x >= MaxX
      return [
        ([x, isgoal(x)], 0.9),
        ([x - StepX, isgoal(x - StepX)], 0.1)]
    elseif x <= MinX
      return [
        ([x, isgoal(x)], 0.2),
        ([x + StepX, isgoal(x + StepX)], 0.8)]
    else
      return [
        ([x, isgoal(x)], 0.1),
        ([x - StepX, isgoal(x - StepX)], 0.1),
        ([x + StepX, isgoal(x + StepX)], 0.8)]
    end
  elseif move == "W"
    if x >= MaxX
      return [
        ([x, isgoal(x)], 0.1),
        ([x - StepX, isgoal(x - StepX)], 0.9)]
    elseif x <= MinX
      return [
        ([x, isgoal(x)], 0.9),
        ([x + StepX, isgoal(x + StepX)], 0.1)]
    else
      return [
        ([x, isgoal(x)], 0.1),
        ([x - StepX, isgoal(x - StepX)], 0.8),
        ([x + StepX, isgoal(x + StepX)], 0.1)]
    end
  elseif move == "stop"
    return [([x, isgoal(x)], 1.0)]
  end
end

mytransition (generic function with 1 method)

In [5]:
transition!(mdp, ["x", "goal", "move"], mytransition)

PLite.LazyFunc(false, ["x", "goal", "move"], mytransition)

In [6]:
function myreward(x::Float64, goal::AbstractString, move::AbstractString)
  if goal == "yes" && move == "stop"
    return 1
  else
    return 0
  end
end

reward!(mdp, ["x", "goal", "move"], myreward)

PLite.LazyFunc(false, ["x", "goal", "move"], myreward)

In [7]:
solver = SerialValueIteration()

SerialValueIteration(true, 1000, 0.0001, 0.99, Dict{AbstractString, PLite.LazyDiscrete}(), Dict{AbstractString, PLite.LazyDiscrete}(), GridInterpolations.RectangleGrid{0}(), GridInterpolations.RectangleGrid{0}())

In [8]:
const StepX = 20
discretize_statevariable!(solver, "x", StepX)

PLite.LazyDiscrete("x", 20.0)

In [9]:
solver = SerialValueIteration(maxiter=40, discount=1, verbose=true)
discretize_statevariable!(solver, "x", StepX)

PLite.LazyDiscrete("x", 20.0)

In [10]:
solution = solve(mdp, solver)

┌ Info: mdp and value iteration solver passed basic checks
└ @ PLite /Users/jeff/.julia/packages/PLite/HjLJ8/src/valueiteration_checks.jl:5


iter 0, resid: 1.0
iter 1, resid: 1.0
iter 2, resid: 1.0
iter 3, resid: 1.0
iter 4, resid: 1.0
iter 5, resid: 1.0
iter 6, resid: 1.0
iter 7, resid: 1.0
iter 8, resid: 1.0
iter 9, resid: 1.0
iter 10, resid: 1.0
iter 11, resid: 1.0
iter 12, resid: 1.0
iter 13, resid: 1.0
iter 14, resid: 1.0
iter 15, resid: 1.0
iter 16, resid: 1.0
iter 17, resid: 1.0
iter 18, resid: 1.0
iter 19, resid: 1.0
iter 20, resid: 1.0
iter 21, resid: 1.0
iter 22, resid: 1.0
iter 23, resid: 1.0
iter 24, resid: 1.0
iter 25, resid: 1.0
iter 26, resid: 1.0
iter 27, resid: 1.0
iter 28, resid: 1.0
iter 29, resid: 1.0
iter 30, resid: 1.0
iter 31, resid: 1.0
iter 32, resid: 1.0
iter 33, resid: 1.0
iter 34, resid: 1.0
iter 35, resid: 1.0
iter 36, resid: 1.0
iter 37, resid: 1.0
iter 38, resid: 1.0
iter 39, resid: 1.0


┌ Warning: maximum number of iterations reached; check accuracy of solutions
└ @ PLite /Users/jeff/.julia/packages/PLite/HjLJ8/src/valueiteration_serial.jl:90
┌ Info: value iteration solution generated
│ cputime [s] = 0.0
│ number of iterations = 40
│ residual = 1.0
└ @ PLite /Users/jeff/.julia/packages/PLite/HjLJ8/src/valueiteration_serial.jl:94


PLite.ValueIterationSolution([36.468750000000036 36.734375000000036 … 38.611111111111114 37.5; 37.34375000000002 38.59375000000001 … 36.861111111111114 36.611111111111114; 36.34375000000004 37.593750000000014 … 38.611111111111114 37.5], GridInterpolations.RectangleGrid{2}([0.0, 20.0, 40.0, 60.0, 80.0, 100.0],[1.0, 2.0],), GridInterpolations.RectangleGrid{1}([1.0, 2.0, 3.0],), 0.0, 40, 1.0)